In [1]:
import dotenv, os
import json
import sqlite3
from datetime import datetime

### Getting data from JSON

In [2]:
def get_dataCollections(file):
    root = f"../../JSON/json/{file}.json"
    new_list = []
    try:
        with open(root, "r", encoding="utf-8") as file:
            datos = json.load(file)

        for data in datos:
            new_list.append(data)
        return new_list
    except FileNotFoundError:
        print(f"❌ Error: File: '{root}' Doesn't exist.")
    except json.JSONDecodeError:
        print(f"❌ Error: File: '{root}' JSON not valid.")
    except Exception as e:
        print(f"❌ Error: {str(e)}")

In [3]:
users = get_dataCollections("users")
countries = get_dataCollections("countries")
companies = get_dataCollections("companies")
products = get_dataCollections("products")
product_items = get_dataCollections("product_items")
bills = get_dataCollections("bills")
bill_items = get_dataCollections("bill_items")
payments = get_dataCollections("payments")
payment_methods = get_dataCollections("payment_methods")

### Creating Database

In [4]:
try:

    # Connect to DB and create a cursor
    connection = sqlite3.connect("upwork.db")
    db = connection.cursor()
    print('DB Init')

    # Write a query and execute it with cursor
    query = 'select sqlite_version();'
    db.execute(query)

    # Fetch and output result
    result = db.fetchall()
    print('SQLite Version is {}'.format(result))


# Handle errors
except sqlite3.Error as error:
    print('Error occurred - ', error)

DB Init
SQLite Version is [('3.45.3',)]


### Creating Tables

In [5]:
# Enable foreign key constraints
try:
    db.execute("PRAGMA foreign_keys = ON")
    
    # Create countries table
    db.execute('''
    CREATE TABLE IF NOT EXISTS countries (
        id TEXT PRIMARY KEY,
        country TEXT NOT NULL,
        nationality TEXT NOT NULL,
        code TEXT NOT NULL
    )
    ''')
    
    # Create users table
    db.execute('''
    CREATE TABLE IF NOT EXISTS users (
        id TEXT PRIMARY KEY,
        username TEXT NOT NULL UNIQUE,
        first_name TEXT,
        last_name TEXT,
        password TEXT NOT NULL,
        email TEXT NOT NULL UNIQUE,
        gender TEXT CHECK(gender IN ('Male', 'Female')),
        country_id TEXT,
        birthday DATE,
        created DATE,
        FOREIGN KEY (country_id) REFERENCES countries(id)
    )
    ''')
    
    # Create companies table
    db.execute('''
    CREATE TABLE IF NOT EXISTS companies (
        id TEXT PRIMARY KEY,
        company_name TEXT NOT NULL,
        tax_id INTEGER,
        sales INTEGER
    )
    ''')
    
    # Create products table
    db.execute('''
    CREATE TABLE IF NOT EXISTS products (
        id TEXT PRIMARY KEY,
        company_id TEXT,
        product_name TEXT NOT NULL,
        description TEXT,
        price DECIMAL(10,2) NOT NULL,
        release DATE,
        department TEXT,
        updated TIMESTAMP,
        FOREIGN KEY (company_id) REFERENCES companies(id)
    )
    ''')
    
    # Create product_items table
    db.execute('''
    CREATE TABLE IF NOT EXISTS product_items (
        id TEXT PRIMARY KEY,
        product_item_id TEXT NOT NULL,
        serial_number TEXT UNIQUE,
        status TEXT CHECK(status IN ('available', 'sold', 'returned')),
        FOREIGN KEY (product_item_id) REFERENCES products(id)
    )
    ''')
    
    # Create payment_methods table
    db.execute('''
    CREATE TABLE IF NOT EXISTS payment_methods (
        id TEXT PRIMARY KEY,
        payment_methods TEXT CHECK(payment_methods IN ('Paypal', 'Binance', 'Zinly', 'Zeller', 'American Express', 'Master Card', 'Visa', 'Bank of America'))
    )
    ''')
    
    # Create bills table
    db.execute('''
    CREATE TABLE IF NOT EXISTS bills (
        id TEXT PRIMARY KEY,
        user_id TEXT NOT NULL,
        bill_address TEXT NOT NULL,
        shipping_address TEXT NOT NULL,
        status TEXT CHECK(status IN ('pending', 'paid', 'shipped', 'delivered', 'cancelled')),
        created DATE,
        FOREIGN KEY (user_id) REFERENCES users(id)
    )
    ''')
    
    # Create payments table
    db.execute('''
    CREATE TABLE IF NOT EXISTS payments (
        id TEXT PRIMARY KEY,
        bill_id TEXT NOT NULL,
        payment_methods_id TEXT NOT NULL,
        transaction_id TEXT,
        status TEXT CHECK(status IN ('pending', 'completed', 'failed', 'refunded')),
        created DATE,
        FOREIGN KEY (bill_id) REFERENCES bills(id),
        FOREIGN KEY (payment_methods_id) REFERENCES payment_methods(id)
    )
    ''')
    
    # Create bill_items table
    db.execute('''
    CREATE TABLE IF NOT EXISTS bill_items (
        id TEXT PRIMARY KEY,
        bill_id TEXT NOT NULL,
        product_item_id TEXT NOT NULL,
        quantity INTEGER NOT NULL DEFAULT 1,
        FOREIGN KEY (bill_id) REFERENCES bills(id),
        FOREIGN KEY (product_item_id) REFERENCES product_items(id)
    )
    ''')

    print(f"Database 'upwork' created successfully with all tables.")
except sqlite3.Error as error:
    print('Error occurred - ', error)

Database 'upwork' created successfully with all tables.


# Inserting into Database

In [6]:
def insert_data(query,data_tuple):
    try:
        db.execute("PRAGMA foreign_keys = ON")

        db.executemany(query,data_tuple)
            
        print(f"Successfully inserted.")
            
    except sqlite3.IntegrityError as e:
        print(f"Integrity error: {e}. Possible duplicate data.")
    except sqlite3.Error as e:
        print(f"Database error: {e}")
    except Exception as e:
        print(f"General error: {e}")

# Getting sql query and data

## Countries

In [7]:
try:
    country_query = """INSERT INTO countries VALUES (?,?,?,?)"""
    country_tuple = [ (
        country["id"],
        country["country"],
        country["nationality"],
        country["code"]
    ) for country in countries]
except Exception as e:
    print(f"Error: {e}")

insert_data(country_query,country_tuple)

Successfully inserted.


## Users

In [8]:
try:
    users_query = """INSERT INTO users VALUES (?,?,?,?,?,?,?,?,?,?)"""
    users_tuple = [ (
        user["id"],
        user["username"],
        user["first_name"],
        user["last_name"],
        user["password"],
        user["email"],
        user["gender"],
        user["country"],
        user["birthday"],
        user["created"]
    ) for user in users]
except Exception as e:
    print(f"Error: {e}")

insert_data(users_query,users_tuple)

Successfully inserted.


## Companies

In [9]:
try:
    companies_query = """INSERT INTO companies VALUES (?,?,?,?)"""
    companies_tuple = [(
        company["id"],
        company["company_name"],
        company["tax_id"],
        company["sales"]
    ) for company in companies]
except Exception as e:
    print(f"Error: {e}")
    
insert_data(companies_query,companies_tuple)

Successfully inserted.


## Products

In [10]:
try:
    products_query = """INSERT INTO products VALUES (?,?,?,?,?,?,?,?)"""
    products_tuple = [(
        product["id"],
        product["company_id"],
        product["product_name"],
        product["description"],
        product["price"],
        product["release"],
        product["department"],
        datetime.now().isoformat()
    ) for product in products]
except Exception as e:
    print(f"Error: {e}")
    
insert_data(products_query,products_tuple)

Successfully inserted.


## Products Items

In [11]:
try:
    product_items_query = """INSERT INTO product_items VALUES (?,?,?,?)"""
    product_items_tuple = [(
        item["id"],
        item["product_item_id"],
        item["serial_number"],
        item["status"]
    ) for item in product_items]
except Exception as e:
    print(f"Error: {e}")

insert_data(product_items_query,product_items_tuple)

Successfully inserted.


## Bills

In [12]:
try:
    bills_query = """INSERT INTO bills VALUES (?,?,?,?,?,?)"""
    bills_tuple = [(
        bill["id"],
        bill["user_id"],
        bill["bill_address"],
        bill["shipping_address"],
        bill["status"],
        bill["created"]
    ) for bill in bills]
except Exception as e:
    print(f"Error: {e}")

insert_data(bills_query,bills_tuple)

Successfully inserted.


## Bills Items

In [13]:
try:
    bill_items_query = """INSERT INTO bill_items VALUES (?,?,?,?)"""
    bill_items_tuple = [(
        bill_item["id"],
        bill_item["bill_id"],
        bill_item["product_item_id"],
        bill_item["quantity"]
    ) for bill_item in bill_items]
except Exception as e:
    print(f"Error: {e}")

insert_data(bill_items_query,bill_items_tuple)

Successfully inserted.


## Payment_methods

In [14]:
try:
    payment_methods_query = """INSERT INTO payment_methods VALUES (?,?)"""
    payment_methods_tuple = [(
        method["id"],
        method["payment_methods"]
    ) for method in payment_methods]
except Exception as e:
    print(f"Error: {e}")

insert_data(payment_methods_query,payment_methods_tuple)

Successfully inserted.


## Payments

In [15]:
try:
    payments_query = """INSERT INTO payments VALUES (?,?,?,?,?,?)"""
    payments_tuple = [(
        payment["id"],
        payment["bill_id"],
        payment["payment_methods_id"],
        payment["transaction_id"],
        payment["status"],
        payment["created"]
    ) for payment in payments]
except Exception as e:
    print(f"Error: {e}")

insert_data(payments_query,payments_tuple)

Successfully inserted.


### Close Database

In [16]:
# Commit changes and close connection
connection.commit()
connection.close()